# Deforestation quantification using classification based object detection 


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os, pathlib

import numpy as np
import time
import datetime
import math


import PIL.Image as Image
import matplotlib.pylab as plt
import matplotlib as mpl


os.environ['TF_USE_LEGACY_KERAS'] = '1'
import tensorflow as tf
import tensorflow_hub as hub

# https://github.com/tensorflow/tensorflow/issues/65419
# import tf_keras
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tf_keras as keras
else:
    keras = tf.keras

from tensorflow.keras import layers

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
tf.config.experimental.set_memory_growth(gpus[0], enable=True)

In [ ]:
input_data_dir = './../../Data/MultiModalGenAI/resisc45/NWPU-RESISC45_small'
output_models_dir = './../../models/MultiModalGenAI/deforestation'
# modelfname= 'deforestation_model_1736702508'#'deforestation_model_1735604427' 
modelfname= 'deforestation_model_1736723402'

batch_size = 32
img_height = 224
img_width = 224


In [ ]:
trained_model = tf.keras.models.load_model(f'{output_models_dir}/{modelfname}')

In [ ]:
dataset_path = pathlib.Path(input_data_dir).with_suffix('')
image_count = len(list(dataset_path.glob('*/*.jpg')))
print(image_count)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  str(dataset_path),
  validation_split=.997,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


In [ ]:
class_names = np.array(test_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for test_image_batch, test_labels_batch in test_ds:
  print(test_image_batch.shape)
  print(test_image_batch.shape)
  break

In [ ]:
unique, counts = np.unique(class_names[test_labels_batch], return_counts=True)
dict(zip(unique, counts))

In [ ]:
test_predictions_batch = trained_model.predict(test_image_batch)

In [ ]:
test_predicted_id = tf.math.argmax(test_predictions_batch, axis=-1)
test_predicted_label_batch = class_names[test_predicted_id]
print(test_predicted_label_batch)


In [ ]:
_ = plt.figure(figsize=(10,12))
_ = plt.subplots_adjust(wspace=.5, hspace=0.5)
for n in range(15):
    _ = plt.subplot(3,5,n+1)
    _ = plt.imshow(test_image_batch[n])
    _ = plt.title(f'Pred:{str(test_predicted_label_batch[n].title())}\nReal:{class_names[test_labels_batch[n]]}')
    _ = plt.axis('off')
    _ = plt.suptitle("Model predictions on Validation Data")

## Patched Object Detection using transfer learning based image classification 


In [ ]:
get_patches = lambda x: (tf.reshape(
    tf.image.extract_patches(
        images=tf.expand_dims(x, 0),
        sizes=[1, patch_height, patch_width, 1],
        strides=[1, stride_height, stride_width, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'), (-1, patch_height, patch_width, 3))
)

def create_heatmap(predicts_ids, patches_order, patches_shape):
    patch_num_y, patch_num_x = patches_order
    patch_height, patch_width = patches_shape
    # heatmap = np.full(patches_shape, np.nan)
    index = 0
    heatmap = np.full(patches_shape, predicts_ids[index])
    index+=1
    
    for col_index in range(patch_num_x-1):
        heatmap = np.concatenate((heatmap,np.full(patches_shape, predicts_ids[index])),axis=1)
        index+=1
        
    for row_index in range(patch_num_y-1):
        rowarray  = np.full(patches_shape, predicts_ids[index])
        index+=1
        for col_index in range(patch_num_x-1):
            rowarray = np.concatenate((rowarray,np.full(patches_shape, predicts_ids[index])),axis=1)
            index+=1
        heatmap = np.concatenate((heatmap, rowarray),axis=0)
        
    return heatmap

# https://keras.io/examples/vision/grad_cam/
def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Use jet colormap to colorize heatmap
    jet = mpl.colormaps["jet"]

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    # jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image, cropped to match the heatmap
    superimposed_img = jet_heatmap * alpha + img[:heatmap.shape[0],:heatmap.shape[1],:]
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    return  tf.keras.utils.load_img(cam_path)


resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(img_height, img_width),
  layers.Rescaling(1./255)
])

In [ ]:
# jet = mpl.colormaps["jet"]
# np.arange(10)
# jet_colors = jet(np.arange(10))
# jet_colors
# jet_colors[:, :3]
# # jet_colors = jet(np.arange(256))[:, :3]

In [ ]:
def object_detection(image_path, classif_model):
    target_img = tf.keras.utils.load_img(image_path)
    target_arr  = tf.keras.utils.img_to_array(target_img)

    patches = get_patches(target_arr)
    patches_predicts = classif_model.predict(resize_and_rescale(patches))
    # patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
    # patches_predicts_label = class_names[patches_predicts_id]
    patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

    patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)
    # classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
    classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
    classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

    grad_cam = save_and_display_gradcam(target_arr, classifHeatmapNorm)
    
    return grad_cam \
    , classifHeatmap \
    , keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2)) \
    , target_img #keras.utils.array_to_img(target_arr)

In [ ]:
patch_width = math.floor(img_width*1)
patch_height = math.floor(img_height*1)
stride_width = patch_width
stride_height = patch_height

In [ ]:
!pwd
!ls -la ./../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/Deforestation10*

In [ ]:
images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'


In [ ]:
# #  8 out of 10
# image_after_f = 'Deforestation02HR_after_07_2017'
# image_before_f = 'Deforestation02HR_before_07_2014'

# # 7 out of 10, before looks great for patches
# image_after_f = 'Deforestation02_02HR_after_07_2017'
# image_before_f = 'Deforestation02_02HR_before_07_2014'


image_after_f = 'Deforestation03HR_after_09_2019'
image_before_f = 'Deforestation03HR_before_06_2018'

In [ ]:
grad_cam_map_a, obj_detection_heatmap_a, deforest_index_map_a, img_a  = object_detection((images_path+image_after_f+'.jpg'), trained_model)
grad_cam_map_a
# deforest_index_map_a
# img_a

In [ ]:
grad_cam_map_b, obj_detection_heatmap_b, deforest_index_map_b, img_b  = object_detection((images_path+image_before_f+'.jpg'), trained_model)
grad_cam_map_b
# deforest_index_map_b
# img_b

### Deforestation quantification

In [ ]:
obj_detection_heatmap_a.shape
obj_detection_heatmap_b.shape

obj_detection_heatmap_a.max()
obj_detection_heatmap_b.max()

obj_detection_heatmap_a.min()
obj_detection_heatmap_b.min()

In [ ]:
diff = np.subtract(obj_detection_heatmap_b, obj_detection_heatmap_a) #np.subtract(1.0, 4.0) -> -3.0

diff.shape
diff.max()
diff.min()

In [ ]:
def norm_heatmap(classifHeatmap):
    maxhp = classifHeatmap.max()
    minhp = classifHeatmap.min()
    return 255.*(classifHeatmap - minhp)/(maxhp - minhp)

nhmb = norm_heatmap(obj_detection_heatmap_b)
nhma = norm_heatmap(obj_detection_heatmap_a)
nhmb.shape
nhmb.max()
nhmb.min()
nhma.shape
nhma.max()
nhma.min()

diff1 = np.subtract(nhmb, nhma)

diff1.shape
diff1.max()
diff1.min()

plt.imshow((diff1 >70) * diff1, interpolation='none')
plt.show()

In [ ]:
hmnorm =  np.uint8(norm_heatmap((diff1 >65) * diff1))
grad_cam = save_and_display_gradcam(tf.keras.utils.img_to_array(img_b), hmnorm)
grad_cam

In [ ]:
image_size_pixels = np.asarray([8192,4320], dtype=np.float32)
fov_meters = np.asarray([6282, 3189], dtype=np.float32)

pixel_size_meters = np.divide(fov_meters, image_size_pixels)
pixel_size_meters